In [1]:
import pandas as pd
df = pd.read_csv("hf://datasets/nateraw/us-accidents/US_Accidents_Dec21_updated.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
df.columns

Index(['ID', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat', 'Start_Lng',
       'End_Lat', 'End_Lng', 'Distance(mi)', 'Description', 'Number', 'Street',
       'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone',
       'Airport_Code', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)',
       'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

Now we need to add more columns to our dataframe to analyze accident trends. Whether they have occured during holidays, which day of the week, etc.. 

In [3]:
import holidays


In [14]:
df.iloc[229886]['Start_Time']

'2021-11-11 19:36:30.000000000'

In [13]:
date_string = "2021-11-11 19:36:30.000000000"
date_object = pd.to_datetime(date_string)
date_object

Timestamp('2016-02-08 00:37:08')

In [34]:
for date, name in sorted(holidays.US(years=2024).items()):
    print(date, name)

2024-01-01 New Year's Day
2024-01-15 Martin Luther King Jr. Day
2024-02-19 Washington's Birthday
2024-05-27 Memorial Day
2024-06-19 Juneteenth National Independence Day
2024-07-04 Independence Day
2024-09-02 Labor Day
2024-10-14 Columbus Day
2024-11-11 Veterans Day
2024-11-28 Thanksgiving
2024-12-25 Christmas Day


In [15]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='mixed') # it has multiple datetime formats hence mixed

# Create new columns for day of the week, date, and year
df['Day_of_Week'] = df['Start_Time'].dt.day_name()      
df['Date'] = df['Start_Time'].dt.date                
df['Year'] = df['Start_Time'].dt.year                  

# List of US holidays
us_holidays = holidays.US()

# Create Holiday flag and Holiday name columns
df['Is_Holiday'] = df['Start_Time'].apply(lambda x: x in us_holidays)
df['Holiday_Name'] = df['Start_Time'].apply(lambda x: us_holidays.get(x) if x in us_holidays else None)


In [30]:
df.loc[df['Is_Holiday'] == True].head(2)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Day_of_Week,Date,Year,Is_Holiday,Holiday_Name
162,A-163,4,2016-02-15 02:01:29,2016-02-15 08:01:29,40.13386,-80.00666,40.12519,-79.95815,2.632,Closed between PA-917/Exit 32 and PA-481/Exit ...,...,False,Night,Night,Night,Night,Monday,2016-02-15,2016,True,Washington's Birthday
163,A-164,4,2016-02-15 03:06:11,2016-02-15 09:06:11,40.43954,-80.00971,40.43420,-79.99635,0.794,Closed between I-279/Exit 70C and Grant St/Exi...,...,False,Night,Night,Night,Night,Monday,2016-02-15,2016,True,Washington's Birthday


In [35]:
df.to_csv('added_columns.csv', index=False)